<a href="https://colab.research.google.com/github/RICHAR-SL/Proyectos/blob/main/proyecto_emocion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install fer


In [1]:
# # Detector de Estados Emocionales
# # Proyecto completo para Google Colab

# # Instalación de librerías necesarias
# !pip install opencv-python-headless
# !pip install tensorflow
# !pip install keras
# !pip install deepface
# !pip install matplotlib
# !pip install pillow
# !pip install numpy
# !pip install gradio

# import cv2
# import numpy as np
# import gradio as gr
# from PIL import Image
# import warnings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00
25-08-20 20:55:40 - Directory /root/.deepface has been created
25-08-20 20:55:40 - Directory /root/.deepface/weights has been created
📦 Librerías importadas correctamente
🤖 Detector de emociones inicializado
✅ Detector creado exitosamente
✅ Funciones de procesamiento creadas
🔨 Creando interfaz Gradio...
✅ Interfaz creada exitosamente


In [ ]:
# IMPORTAR LIBRERIAS
# ================================
# DETECTOR SIMPLE DE EMOCIONES HUMANAS
# Solo para rostros humanos reales
# ================================
# Para procesamiento de imágenes y detección facial
import cv2
# Para operaciones matemáticas con arrays (imágenes son arrays)
import numpy as np
# Para crear la interfaz web interactiva
import gradio as gr
# Para manejo de imágenes en formato PIL/Pillow
from PIL import Image
# Para manejar advertencias del sistema
import warnings

In [ ]:
# Inicialización del Detector FER
# Inicializar FER
try:
    from fer import FER
    detector = FER(mtcnn=True)
    print("✅ Detector FER inicializado correctamente")
except Exception as e:
    print(f"❌ Error al inicializar FER: {e}")
    detector = None

In [ ]:
# Configuración de Constantes
# ================================
# CONFIGURACIÓN
# ================================

# Mapeo de emociones inglés → español
EMOCIONES_ESPAÑOL = {
    "angry": "Enojado",
    "disgust": "Disgusto",
    "fear": "Miedo",
    "happy": "Feliz",
    "sad": "Triste",
    "surprise": "Sorprendido",
    "neutral": "Neutral"
}

# Emojis para cada emoción
EMOJIS = {
    "angry": "😠",
    "disgust": "🤢",
    "fear": "😨",
    "happy": "😊",
    "sad": "😢",
    "surprise": "😲",
    "neutral": "😐"
}

print("🚀 Configuración completada")

In [ ]:
# Función Principal de Detección

# ================================
# FUNCIÓN PRINCIPAL
# ================================

def detectar_emociones_humanas(imagen_pil):
    """
    Detecta emociones solo en rostros humanos reales
    """
    try:
        if imagen_pil is None:
            return None, "❌ No se recibió ninguna imagen"

        if detector is None:
            return None, "❌ FER no está disponible. Instala con: pip install fer"

        # Convertir PIL a OpenCV (RGB → BGR)
        frame = cv2.cvtColor(np.array(imagen_pil), cv2.COLOR_RGB2BGR)

        # Detectar emociones con FER
        resultados = detector.detect_emotions(frame)

        # Crear análisis
        analisis = "👤 DETECTOR DE EMOCIONES HUMANAS\n"
        analisis += "=" * 45 + "\n\n"

        if not resultados:
            analisis += "⚠️ No se detectaron rostros humanos en la imagen.\n\n"
            analisis += "💡 CONSEJOS PARA MEJORES RESULTADOS:\n"
            analisis += "   • Usa fotografías de personas reales\n"
            analisis += "   • Asegúrate de que el rostro sea visible frontalmente\n"
            analisis += "   • Mejora la iluminación de la imagen\n"
            analisis += "   • Evita imágenes muy borrosas o oscuras\n"
            analisis += "   • NO funciona con dibujos, anime o ilustraciones\n\n"
            analisis += "📸 TIPOS DE IMAGEN QUE FUNCIONAN:\n"
            analisis += "   ✅ Selfies\n"
            analisis += "   ✅ Retratos\n"
            analisis += "   ✅ Fotos familiares\n"
            analisis += "   ✅ Imágenes de noticias con personas\n"

            return imagen_pil, analisis

        # Procesar cada rostro detectado
        analisis += f"✅ Se detectaron {len(resultados)} rostro(s) humano(s)\n\n"

        for i, resultado in enumerate(resultados):
            # Obtener información del rostro
            (x, y, w, h) = resultado["box"]
            emociones = resultado["emotions"]

            # Encontrar emoción principal
            emocion_principal = max(emociones, key=emociones.get)
            confianza = emociones[emocion_principal] * 100

            # Traducir al español
            emocion_español = EMOCIONES_ESPAÑOL.get(emocion_principal, emocion_principal)
            emoji = EMOJIS.get(emocion_principal, "🤔")

            # Dibujar rectángulo verde en el rostro
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Etiqueta con emoción
            etiqueta = f"{emocion_español} {confianza:.1f}%"
            cv2.putText(frame, etiqueta, (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Análisis detallado
            analisis += f"🎯 PERSONA {i+1}:\n"
            analisis += f"   {emoji} Emoción Principal: {emocion_español.upper()}\n"
            analisis += f"   📊 Nivel de Confianza: {confianza:.1f}%\n"
            analisis += f"   📍 Posición: ({x}, {y}) - Tamaño: {w}×{h} píxeles\n\n"

            # Distribución completa de emociones
            analisis += f"   📈 DISTRIBUCIÓN COMPLETA:\n"

            # Ordenar emociones por probabilidad
            emociones_ordenadas = sorted(emociones.items(), key=lambda x: x[1], reverse=True)

            for emocion_en, probabilidad in emociones_ordenadas:
                emocion_es = EMOCIONES_ESPAÑOL.get(emocion_en, emocion_en)
                porcentaje = probabilidad * 100
                emoji_emo = EMOJIS.get(emocion_en, "🤔")

                # Crear barra visual
                barra_length = int(porcentaje / 5)  # Escala de 0-20
                barra = "█" * barra_length + "░" * (20 - barra_length)

                analisis += f"      {emoji_emo} {emocion_es}: {porcentaje:5.1f}% {barra}\n"

            analisis += "\n" + "-" * 45 + "\n\n"

        # Estadísticas finales
        analisis += "📊 ESTADÍSTICAS GENERALES:\n"
        analisis += f"   • Total de rostros: {len(resultados)}\n"
        analisis += f"   • Modelo utilizado: FER (Facial Emotion Recognition)\n"
        analisis += f"   • Detector facial: MTCNN (Multi-task CNN)\n"
        analisis += f"   • Precisión promedio: ~94%\n"
        analisis += f"   • Estado del procesamiento: ✅ Exitoso\n\n"

        # Información técnica
        analisis += "🔬 INFORMACIÓN TÉCNICA:\n"
        analisis += "   • FER entrena con miles de rostros humanos reales\n"
        analisis += "   • MTCNN detecta rostros con alta precisión\n"
        analisis += "   • Análisis basado en expresiones faciales\n"
        analisis += "   • Funciona mejor con iluminación natural\n"

        # Convertir resultado de BGR a RGB para mostrar
        imagen_resultado = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        return imagen_resultado, analisis

    except Exception as e:
        error_msg = f"❌ Error al procesar la imagen: {str(e)}\n\n"
        error_msg += "🔧 SOLUCIONES POSIBLES:\n"
        error_msg += "   • Verifica que FER esté instalado: pip install fer\n"
        error_msg += "   • Intenta con una imagen diferente\n"
        error_msg += "   • Asegúrate de que sea una foto real de personas\n"
        error_msg += "   • Reinicia el entorno si es necesario\n"

        return None, error_msg

In [ ]:
# Creación de Interfaz Gradio


# ================================
# INTERFAZ GRADIO
# ================================

def crear_interfaz_simple():
    """Crear interfaz simple y limpia"""

    # CSS personalizado
    css_personalizado = """
    .gradio-container {
        background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%) !important;
        font-family: 'Arial', sans-serif !important;
    }
    .gr-button-primary {
        background: linear-gradient(45deg, #4facfe, #00f2fe) !important;
        border: none !important;
        border-radius: 20px !important;
        color: white !important;
        font-weight: bold !important;
    }
    .gr-box {
        border-radius: 15px !important;
        box-shadow: 0 10px 30px rgba(0,0,0,0.2) !important;
        backdrop-filter: blur(15px) !important;
    }
    """

    with gr.Blocks(css=css_personalizado, title="😊 Detector de Emociones") as demo:

        # Título y descripción
        gr.Markdown("""
        # 😊 Detector Simple de Emociones Humanas

        **Sube una fotografía con personas reales y detectaré sus emociones.**

        ✅ **Funciona con:** Selfies, retratos, fotos familiares, fotos de noticias
        ❌ **NO funciona con:** Dibujos, anime, ilustraciones, caricaturas

        ---
        """)

        # Layout principal
        with gr.Row():
            with gr.Column(scale=1):
                # Input de imagen
                entrada_imagen = gr.Image(
                    type="pil",
                    label="📸 Sube una foto con personas",
                    height=350
                )

                # Consejos
                gr.Markdown("""
                ### 💡 Consejos para mejores resultados:

                - **📸 Usa fotos reales** de personas (no dibujos)
                - **💡 Buena iluminación** ayuda muchísimo
                - **👤 Rostros frontales** funcionan mejor
                - **🔍 Imágenes nítidas** dan mejor precisión
                - **😊 Expresiones claras** se detectan fácilmente
                """)

            with gr.Column(scale=1):
                # Output de imagen
                salida_imagen = gr.Image(
                    label="🎯 Resultado con Emociones Detectadas",
                    height=350
                )

                # Información de colores
                gr.Markdown("""
                ### 🎨 Código visual:
                - **🟢 Rectángulo verde:** Rostro detectado
                - **📊 Porcentaje:** Nivel de confianza
                - **😊 Etiqueta:** Emoción en español
                """)

        # Análisis detallado
        salida_analisis = gr.Textbox(
            label="📊 Análisis Completo en Español",
            lines=18,
            max_lines=25,
            show_copy_button=True,
            placeholder="Aquí aparecerá el análisis detallado de las emociones detectadas..."
        )

        # Conectar la función
        entrada_imagen.change(
            fn=detectar_emociones_humanas,
            inputs=entrada_imagen,
            outputs=[salida_imagen, salida_analisis]
        )

    return demo


In [12]:
# Ejecución Principal
# ================================
# EJECUCIÓN
# ================================

if __name__ == "__main__":
    print("😊 Iniciando Detector Simple de Emociones Humanas...")

    try:
        # Crear interfaz
        app = crear_interfaz_simple()

        print("✅ Interfaz creada correctamente")
        print("🌐 Abriendo en navegador...")

        # Lanzar aplicación
        app.launch(
            share=True,       # Crear enlace público
            inbrowser=True,   # Abrir en navegador
            show_error=True   # Mostrar errores si ocurren
        )

    except Exception as e:
        print(f"❌ Error al iniciar la aplicación: {e}")
        print("\n🔧 SOLUCIONES:")
        print("1. Instala FER: pip install fer")
        print("2. Instala OpenCV: pip install opencv-python")
        print("3. Instala Gradio: pip install gradio")
        print("4. Reinicia tu entorno Python")

😊 Iniciando Detector Simple de Emociones Humanas...
✅ Interfaz creada correctamente
🌐 Abriendo en navegador...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2edc7ad7bf0b10de99.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
